In [3]:
import numpy as np
import pygame
from datetime import datetime

#set up game area layout
# By changing the block_size other aspects will scale
block_size = 30    # Size of side of square
blocks_w = 10      # game width (in blocks)
blocks_h = 20      # game height (in blocks)
border_w = 500
border_h = 100

play_w = blocks_w * block_size  # Width is 10 blocks
play_h = blocks_h * block_size  # Height is 20 blocks
full_w = 2 * border_w + play_w
full_h = 2 * border_h + play_h + 50
top_left_x = border_w
top_left_y = 2 * border_h
start_x = int(blocks_w / 2)

pygame.font.init()

# Four block shapes: I, J, L, O, S, T, Z
# [['.....','.....','.....','.....','.....']]
#I
I = [['..0..',
      '..0..',
      '..0..',
      '..0..',
      '.....'],
     ['.....',
      '0000.',
      '.....',
      '.....',
      '.....']]
#J
J = [['.....',
      '.0...',
      '.000.',
      '.....',
      '.....'],
     ['.....',
      '..00.',
      '..0..',
      '..0..',
      '.....'],
     ['.....',
      '.....',
      '.000.',
      '...0.',
      '.....'],
     ['.....',
      '..0..',
      '..0..',
      '.00..',
      '.....']]
#L
L = [['.....',
      '.....',
      '.000.',
      '.0...',
      '.....'],
     ['.....',
      '.00..',
      '..0..',
      '..0..',
      '.....'],
     ['.....',
      '...0.',
      '.000.',
      '.....',
      '.....'],
     ['.....',
      '..0..',
      '..0..',
      '..00.',
      '.....']]
#O
O = [['.....',
      '.....',
      '.00..',
      '.00..',
      '.....']]
#S
S = [['.....',
      '..0..',
      '..00.',
      '...0.',
      '.....'],
     ['.....',
      '.....',
      '..00.',
      '.00..',
      '.....']]
#T
T = [['.....',
      '..0..',
      '.000.',
      '.....',
      '.....'],
     ['.....',
      '..0..',
      '..00.',
      '..0..',
      '.....'],
     ['.....',
      '.....',
      '.000.',
      '..0..',
      '.....'],
     ['.....',
      '..0..',
      '.00..',
      '..0..',
      '.....']]
#Z
Z = [['.....',
      '.....',
      '.00..',
      '..00.',
      '.....'],
     ['.....',
      '..0..',
      '.00..',
      '.0...',
      '.....']]
shapes = [I, J, L, O, S, T, Z]
shapes_color = [(255,0,0),(0,255,0),
                (0,0,255),(0,255,255),
                (255,0,255),(255,255,0),
                (128,0,128)]

class Piece(object):
    def __init__ (self, x, y, shape):
        self.x = x
        self.y = y
        self.shape = shape
        self.color = shapes_color[shapes.index(shape)]
        self.rotation = int(np.random.rand()*len(shape))

def create_grid(locked_pos = {}):
    grid = [[(0,0,0) for _ in range(blocks_w)] for _ in range(blocks_h)]
    for hgt in range(blocks_h):
        for wid in range(blocks_w):
            if (wid, hgt) in locked_pos:
                clr = locked_pos[(wid,hgt)]
                grid[hgt][wid] = clr
    return grid

def convert_shape_fmt(this_shape):
    posit = []
    fmt = this_shape.shape [this_shape.rotation % len(this_shape.shape)]
    for i, line in enumerate(fmt):
        row = list(line)
        for j, col in enumerate(row):
            if col == '0':
                posit.append((int(this_shape.x + j), int(this_shape.y + i)))
                             
    for i, pos in enumerate(posit):
        posit[i] = (int(pos[0]-2), int(pos[1]-4))
    return posit

def valid_space(grid, piece):
    accepted_pos = [[(j,i) for j in range (blocks_w) if grid[i][j] == (0,0,0)] for i in range (blocks_h)]
    accepted_pos = [j for sub in accepted_pos for j in sub]
    formatted = convert_shape_fmt(piece)
    for pos in formatted:
        if pos not in accepted_pos:
            if pos[1] > -1:
                return False
    return True

def check_lost(positions):
    for pos in positions:
        x, y = pos
        if y < -1:
            return True
    return False

def get_shape():
    return Piece(start_x, 0, np.random.choice(shapes))

def draw_text_middle(surface, text, size, color):
    font = pygame.font.SysFont('Arial',size, bold = True)
    bolded = font.render(text, 1, color)
    surface.blit(bolded,((full_w-bolded.get_width())/2, (full_h-bolded.get_height())/2))

def draw_grid(surface, grid):
    for hgt in range(blocks_h+1):
        pygame.draw.line(surface,(128,128,128), 
                         (top_left_x, top_left_y + hgt*block_size), 
                         (top_left_x + blocks_w*block_size, top_left_y + hgt*block_size))
    for wid in range(blocks_w+1):
        pygame.draw.line(surface,(128,128,128), 
                         (top_left_x + wid*block_size, top_left_y), 
                         (top_left_x + wid*block_size, top_left_y + blocks_h*block_size))


def clear_rows(grid, locked):

    inc = 0
    # backward (bottom to top scan)
    for i in range(len(grid)-1, -1, -1):
        row = grid[i]
        if (0, 0, 0) not in row:
            inc += 1
            for j in range(len(row)):
                del locked[(j,i)]
        else:
            if inc > 0:
                k = i+inc
                for j in range(len(row)):
                    if (j,i) in locked:
                        locked[(j,k)] = locked[(j,i)]
                        del locked[(j,i)]

    return inc, locked

def draw_next_shape(surface, shp):
    font = pygame.font.SysFont('Arial',60)
    next_s = font.render('Next Shape:',1, (255,255,255))
    next_x = top_left_x + play_w + 150
    next_y = top_left_y + play_h/2 - 150
    fmt = shp.shape[shp.rotation]
    
    for i, line in enumerate(fmt):
        row = list(line)
        for j,col in enumerate(row):
            if col == '0':
                pygame.draw.rect(surface, shp.color, (next_x + j*block_size, 
                                 next_y + i*block_size, 
                                 block_size, block_size), 0)
    surface.blit(next_s, (next_x - 50, next_y - 100))

def add_score(score):
    now = datetime.now()
    dt_string = now.strftime("%Y/%m/%d %H:%M:%S\n")
    with open('data/score_log','a') as f:
        f.writelines(str(int(score))+','+dt_string)

def draw_window(surface, grid, score):
    surface.fill((0,0,0))
    pygame.font.init()
    font = pygame.font.SysFont('Arial',60)
    title = font.render('Tetris',1, (255,255,255))
    surface.blit(title,(full_w/2 - title.get_width()/2, 20))
    
    font = pygame.font.SysFont('Arial',60)
    score_s = font.render('Score:',1, (255,255,255))
    score_x = 50
    score_y = top_left_y + 50
    surface.blit(score_s, (score_x, score_y))
    score_s = font.render(str(score),1, (255,255,255))
    score_x = 100
    score_y += 100
    surface.blit(score_s, (score_x, score_y))
    
    for hgt in range(blocks_h):
        for wid in range(blocks_w):
            pygame.draw.rect(surface, grid[hgt][wid],
                             (top_left_x + wid*block_size, 
                            top_left_y + hgt*block_size, 
                            block_size, block_size), 0)
    pygame.draw.rect(surface, (255,0,0), (top_left_x, top_left_y, play_w, play_h), 4)
    draw_grid(surface, grid)
#     pygame.display.update()

def main(win):
    locked_blocks = {}
    grid = create_grid(locked_blocks)
    
    change_piece = False
    run = True
    next_piece = get_shape()
    curr_piece = get_shape()
    
    clock = pygame.time.Clock()
    fall_time = 0
    fall_speed =.27
    level_time = 0
    score = 0
    
    while run:
        fall_time += clock.get_rawtime()
        level_time += clock.get_rawtime()
        clock.tick()
        
        if level_time/1000 >5:
            level_time = 0
            if fall_speed > .12:
                fall_speed -= .005
        
        if fall_time/1000 > fall_speed:
            fall_time = 0
            curr_piece.y += 1
            grid = create_grid(locked_blocks)
            if not (valid_space(grid, curr_piece)) and (curr_piece.y > 0):
                curr_piece.y -= 1
                change_piece = True
        
        grid = create_grid(locked_blocks)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.display.quit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    curr_piece.x -= 1
                    if not (valid_space(grid, curr_piece)):
                        curr_piece.x += 1
                if event.key == pygame.K_RIGHT:
                    curr_piece.x += 1
                    if not (valid_space(grid, curr_piece)):
                        curr_piece.x -= 1
                if event.key == pygame.K_DOWN:
                    while valid_space(grid, curr_piece):
                        curr_piece.y += 1
                    curr_piece.y -= 1
                if event.key == pygame.K_UP:
                    curr_piece.rotation += 1 
                    curr_piece.rotation %= len(curr_piece.shape)
                    if not (valid_space(grid, curr_piece)):
                        if curr_piece.rotation == 0:
                            curr_piece.rotation = len(curr_piece.shape) - 1
                        else:
                            curr_piece.rotation -= 1
        shape_pos = convert_shape_fmt(curr_piece)
        for i in range(len(shape_pos)):
            x, y = shape_pos[i]
            if y > -1:
                grid[y][x] = curr_piece.color
        if change_piece:
            for pos in shape_pos:
                p = (pos[0],pos[1])
                locked_blocks[p] = curr_piece.color
            curr_piece = next_piece
            next_piece = get_shape()
            change_piece = False
            increment, new_block_set = clear_rows(grid, locked_blocks)
            score += increment * 10
            locked_blocks = new_block_set

        draw_window(win, grid, score)
        draw_next_shape(win, next_piece)
        pygame.display.update()

        if check_lost(locked_blocks):
            draw_text_middle(win, "You Lost!", 80, (255, 255, 255))
            pygame.display.update()
            pygame.time.delay(2000)
            run = False
            add_score(score)

def main_menu(win):
    run = True
    while run:
        win.fill((0, 0, 0))
        draw_text_middle(win,"Press any key to Start", 60, (255, 255, 255))
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            if event.type == pygame.KEYDOWN:
                main(win)

    pygame.display.quit()

win = pygame.display.set_mode((full_w, full_h))
pygame.display.set_caption('Tetris')
main_menu(win)

KeyboardInterrupt: 